<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/objects/object_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# `ObjectIndex` 类

`ObjectIndex` 类允许对任意的 Python 对象进行索引。因此，它非常灵活，适用于各种用例。例如：
- [使用 `ObjectIndex` 对工具对象进行索引，然后由代理使用。](https://docs.llamaindex.ai/en/stable/examples/agent/openai_agent_retrieval.html#building-an-object-index)
- [使用 `ObjectIndex` 对 SQLTableSchema 对象进行索引。](https://docs.llamaindex.ai/en/stable/examples/index_structs/struct_indices/SQLIndexDemo.html#part-2-query-time-retrieval-of-tables-for-text-to-sql)

要构建一个 `ObjectIndex`，我们需要一个索引以及另一个抽象，即 `ObjectNodeMapping`。正如其名称所示，该映射提供了在节点和相关对象之间以及反之间进行转换的方法。另外，还存在一个 `from_objects()` 类方法，可以方便地从一组对象构建一个 `ObjectIndex`。

在这个笔记本中，我们将快速介绍如何使用 `SimpleObjectNodeMapping` 构建一个 `ObjectIndex`。


In [ ]:
from llama_index.core import Settings

Settings.embed_model = "local"

In [ ]:
from llama_index.core import VectorStoreIndexfrom llama_index.core.objects import ObjectIndex, SimpleObjectNodeMapping# 一些非常随意的对象obj1 = {"input": "嘿，最近怎么样"}obj2 = ["a", "b", "c", "d"]obj3 = "llamaindex 是一个很棒的库！"arbitrary_objects = [obj1, obj2, obj3]# （可选）对象-节点映射obj_node_mapping = SimpleObjectNodeMapping.from_objects(arbitrary_objects)nodes = obj_node_mapping.to_nodes(arbitrary_objects)# 对象索引object_index = ObjectIndex(    index=VectorStoreIndex(nodes=nodes),    object_node_mapping=obj_node_mapping,)# 从对象创建对象索引（默认 index_cls=VectorStoreIndex）object_index = ObjectIndex.from_objects(    arbitrary_objects, index_cls=VectorStoreIndex)

### 作为检索器
有了`object_index`，我们可以将其用作检索器，针对索引对象进行检索。


In [ ]:
object_retriever = object_index.as_retriever(similarity_top_k=1)
object_retriever.retrieve("llamaindex")

['llamaindex is an awesome library!']

我们还可以将节点后处理器添加到对象索引检索器中，以便轻松方便地进行重新排列等操作。


In [ ]:
%pip install llama-index-postprocessor-colbert-rerank

In [ ]:
from llama_index.postprocessor.colbert_rerank import ColbertRerank

retriever = object_index.as_retriever(
    similarity_top_k=2, node_postprocessors=[ColbertRerank(top_n=1)]
)
retriever.retrieve("a random list object")

['llamaindex is an awesome library!']

## 使用存储集成（例如Chroma）

对象索引支持与LlamaIndex中的任何现有存储后端进行集成。

以下部分将介绍如何使用`Chroma`作为示例来设置集成。


In [ ]:
%pip install llama-index-vector-stores-chroma

In [ ]:
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart2")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

object_index = ObjectIndex.from_objects(
    arbitrary_objects,
    index_cls=VectorStoreIndex,
    storage_context=storage_context,
)

FileNotFoundError: [Errno 2] No such file or directory: './chroma_db2'

In [ ]:
object_retriever = object_index.as_retriever(similarity_top_k=1)
object_retriever.retrieve("llamaindex")

['llamaindex is an awesome library!']

现在，让我们“重新加载”索引。


In [ ]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

object_index = ObjectIndex.from_objects_and_index(arbitrary_objects, index)

In [ ]:
object_retriever = object_index.as_retriever(similarity_top_k=1)
object_retriever.retrieve("llamaindex")

['llamaindex is an awesome library!']

请注意，当我们重新加载索引时，仍然需要传递对象，因为这些对象并未保存在实际的索引/向量数据库中。


## [高级] 自定义映射

对于特殊情况，您希望完全控制对象如何映射到节点，还可以提供 `to_node_fn()` 和 `from_node_fn()` 钩子函数。

当您要转换特殊对象或希望在运行时动态创建对象而不是将它们保存在内存中时，这将非常有用。

下面是一个小例子。


In [ ]:
from llama_index.core.schema import TextNode

my_objects = {
    str(hash(str(obj))): obj for i, obj in enumerate(arbitrary_objects)
}


def from_node_fn(node):
    return my_objects[node.id]


def to_node_fn(obj):
    return TextNode(id=str(hash(str(obj))), text=str(obj))


object_index = ObjectIndex.from_objects(
    arbitrary_objects,
    index_cls=VectorStoreIndex,
    from_node_fn=from_node_fn,
    to_node_fn=to_node_fn,
)

object_retriever = object_index.as_retriever(similarity_top_k=1)

object_retriever.retrieve("llamaindex")

['llamaindex is an awesome library!']

## 使用对象持久化`ObjectIndex`到磁盘

在持久化`ObjectIndex`时，我们需要处理索引以及对象节点映射。持久化索引很简单，可以通过通常的方式处理（例如，参见这个[指南](https://docs.llamaindex.ai/en/stable/module_guides/storing/save_load.html#persisting-loading-data)）。然而，当涉及持久化`ObjectNodeMapping`时情况就有些不同了。由于我们正在使用`ObjectIndex`对任意Python对象进行索引，可能会出现这样的情况（也许比我们想象的更常见），即这些任意对象是不可序列化的。在这种情况下，您可以持久化索引，但用户必须维护一种方法来重新构建`ObjectNodeMapping`，以便能够重新构建`ObjectIndex`。为了方便起见，`ObjectIndex`上有`persist`和`from_persist_dir`方法，它们将尝试持久化和加载先前保存的`ObjectIndex`。


### 快乐的例子


In [ ]:
# 持久化到磁盘（如果未提供路径，则持久化到默认路径./storage）object_index.persist()

In [ ]:
# 重新加载（如果未提供路径，则尝试从默认路径./storage加载）reloaded_object_index = ObjectIndex.from_persist_dir()

In [ ]:
reloaded_object_index._object_node_mapping.obj_node_mapping

{7981070310142320670: {'input': "Hey, how's it going"},
 -5984737625581842527: ['a', 'b', 'c', 'd'],
 -8305186196625446821: 'llamaindex is an awesome library!'}

In [ ]:
object_index._object_node_mapping.obj_node_mapping

{7981070310142320670: {'input': "Hey, how's it going"},
 -5984737625581842527: ['a', 'b', 'c', 'd'],
 -8305186196625446821: 'llamaindex is an awesome library!'}

### 无法正常工作的示例


In [ ]:
from llama_index.core.tools import FunctionToolfrom llama_index.core import SummaryIndexfrom llama_index.core.objects import SimpleToolNodeMappingdef add(a: int, b: int) -> int:    """将两个整数相加并返回结果整数"""    return a + bdef multiply(a: int, b: int) -> int:    """将两个整数相乘并返回结果整数"""    return a * bmultiply_tool = FunctionTool.from_defaults(fn=multiply)add_tool = FunctionTool.from_defaults(fn=add)object_mapping = SimpleToolNodeMapping.from_objects([add_tool, multiply_tool])object_index = ObjectIndex.from_objects(    [add_tool, multiply_tool], object_mapping)

In [ ]:
# 尝试直接持久化对象映射将会引发错误object_mapping.persist()

NotImplementedError: Subclasses should implement this!

In [ ]:
# 尝试在这里持久化对象索引，将向用户抛出警告。 object_index.persist()

/var/folders/0g/wd11bmkd791fz7hvgy1kqyp00000gn/T/ipykernel_77363/46708458.py:2: UserWarning: Unable to persist ObjectNodeMapping. You will need to reconstruct the same object node mapping to build this ObjectIndex
  object_index.persist()


**在这种情况下，只有索引被持久化了。** 为了按照上面提到的重新构建`ObjectIndex`，我们需要手动重新构建`ObjectNodeMapping`，并将其提供给`ObjectIndex.from_persist_dir`方法。


In [ ]:
reloaded_object_index = ObjectIndex.from_persist_dir(    object_node_mapping=object_mapping  # 如果没有这个，就会抛出错误)